# 🚀 OpenAI Python SDK 101

In this notebook we’ll learn how to interact with Large Language Models (LLMs) directly using the **OpenAI Python SDK**.  
This is the **first time** we’re exploring API interactions, so we’ll build up gradually:

1. **Initialize** the client with your API key.  
2. **Minimal call** to the API (Responses API).  
3. Use **Chat Completions** for system + user roles.  
4. Explore **temperature** (randomness) and **top_p** (nucleus sampling).  
5. Add **system prompts** to guide behavior.  
6. Try **streaming tokens** (like ChatGPT typing).  
7. Get **JSON/structured outputs** with schemas.  
8. Handle **errors, timeouts, and retries** gracefully.

By the end, you’ll know how to **call an LLM safely and flexibly** using just the OpenAI SDK.

In [66]:
import os, json, time
from typing import Any, Dict

### API key
- Set your OpenAI API key as an environment variable:  
  `export OPENAI_API_KEY="sk-..."` (macOS/Linux) or `setx OPENAI_API_KEY "sk-..."` (Windows, new terminal required).  
- In Colab: use `os.environ["OPENAI_API_KEY"] = "..."` (for demos only).


In [67]:
# OpenAI Python SDK v1 style
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [68]:
# Minimal "Responses API" call (recommended by OpenAI for new projects)
# Docs: https://platform.openai.com/docs/guides/text  and Responses vs Chat Completions
resp = client.responses.create(
    model="gpt-4o-mini",  # choose any available text-capable model
    input="In one sentence, explain the difference between temperature and top_p for sampling."
)
print(resp.output_text)

Temperature controls the randomness of predictions in sampling, affecting how diverse the outputs are, while top_p (nucleus sampling) limits the choices to a certain cumulative probability, ensuring that only the most likely words are considered for selection.


In [69]:
resp

Response(id='resp_0c95f9131e39d43b0068f337079c588198b0e1cac18930c271', created_at=1760769799.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_0c95f9131e39d43b0068f3370853608198a3d01b31b018a9e1', content=[ResponseOutputText(annotations=[], text='Temperature controls the randomness of predictions in sampling, affecting how diverse the outputs are, while top_p (nucleus sampling) limits the choices to a certain cumulative probability, ensuring that only the most likely words are considered for selection.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summ

In [70]:
# Using Chat Completions (still widely used & supported)
chat = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a concise teaching assistant."},
        {"role": "user", "content": "Give me 3 bullet points about overfitting."}
    ]
)
print(chat.choices[0].message.content)

- **Definition**: Overfitting occurs when a machine learning model learns the training data too well, capturing noise and outliers, which leads to poor performance on unseen data.

- **Symptoms**: A model exhibiting overfitting usually shows very low training error but high validation or test error, indicating that it has memorized the training data instead of generalizing from it.

- **Prevention Techniques**: Common strategies to prevent overfitting include using cross-validation, simplifying the model (e.g., reducing complexity), employing regularization techniques (like L1 or L2), and using dropout in neural networks.


In [71]:
chat

ChatCompletion(id='chatcmpl-CRuwrkBLaqjVVmDqLdPS2FxGZBms9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- **Definition**: Overfitting occurs when a machine learning model learns the training data too well, capturing noise and outliers, which leads to poor performance on unseen data.\n\n- **Symptoms**: A model exhibiting overfitting usually shows very low training error but high validation or test error, indicating that it has memorized the training data instead of generalizing from it.\n\n- **Prevention Techniques**: Common strategies to prevent overfitting include using cross-validation, simplifying the model (e.g., reducing complexity), employing regularization techniques (like L1 or L2), and using dropout in neural networks.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760769801, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', s

In [72]:
messages = [
    {"role": "system", "content": "You are a Python tutor who answers with short code examples."},
    {"role": "user", "content": "Show how to reverse a string in Python."}
]
r = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=2) #High temperature for more creative output
print(r.choices[0].message.content)

You can reverse a string in Python using slicing. Here's a simple example:

```python
original_string = "Hello, World!"
reversed_string = original_string[::-1]
print(reversed_string)
```

Output:
```
!dlroW ,olleH
```


In [73]:
from sys import stdout

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Write a short story about a cat and a dog."}],
    temperature=1,
    stream=True,
)

for event in stream:
    if hasattr(event, "choices"):
        delta = event.choices[0].delta
        if delta and delta.content:
            stdout.write(delta.content)
stdout.write("\n")

Once in a quaint little town, there lived a plucky cat named Whiskers and a jovial dog named Buster. Whiskers, with her sleek gray fur and shimmering green eyes, prided herself on her finesse. She spent her days lounging atop a sun-drenched fence, watching the world below with a dispassionate grace. Buster, on the other hand, was a fluffy golden retriever with a heart as big as his bark. He loved nothing more than chasing butterflies and making friends with anyone he met.

Despite their differences, Whiskers and Buster shared a common ground—the sprawling, vibrant park on the edge of town. Every afternoon, while the sun cast golden rays on the grass, they would meet at their usual spot under the grand old oak tree. Although they often looked at each other with wariness, an unspoken pact had formed between them, and they relished the comfort of each other's presence.

One sunny afternoon, an unusual rustle in the bushes caught Whiskers’ keen ear. “Do you hear that, Buster?” she asked, h

1

In [74]:
message = """I have bought 3 kg of Rice, 4 kg of Dhal, 3 packets of Biscuits, 2 kg of Suger, Format this as a list of json objects with each JSON object in the format:
{
"item": "<item name>", 
"quantity": <quantity in kg or packets> 
}

DO NOT include anything else in the response"""


completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    temperature=1,
    messages=[
        {"role": "user", "content": message}
    ],
)

print(completion.choices[0].message.content)

[
    {
        "item": "Rice", 
        "quantity": 3 
    },
    {
        "item": "Dhal", 
        "quantity": 4 
    },
    {
        "item": "Biscuits", 
        "quantity": 3 
    },
    {
        "item": "Sugar", 
        "quantity": 2 
    }
]


In [75]:
import json

response = completion.choices[0].message.content

items = json.loads(response)
type(items)


list

In [76]:
for item in items:
    print(item["item"])

Rice
Dhal
Biscuits
Sugar


In [77]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI


class Summary(BaseModel):
    topic: str
    key_points: List[str]


client = OpenAI()

completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    temperature=0,
    response_format=Summary,
    messages=[
        {"role": "user", "content": "Topic: Transformers in NLP. Give 3 key points."}
    ],
)

parsed = completion.choices[0].message.parsed
parsed

Summary(topic='Transformers in NLP', key_points=['Transformers utilize self-attention mechanisms to weigh the importance of different words in a sentence, allowing for better context understanding.', 'They enable parallel processing of data, significantly speeding up training times compared to previous sequential models like RNNs and LSTMs.', 'Transformers have led to the development of powerful pre-trained models (e.g., BERT, GPT) that can be fine-tuned for various NLP tasks, achieving state-of-the-art results.'])